In [11]:
import scipy.io as sio
import numpy as np
import os
import sys
from PIL import Image
import errno
from sklearn.decomposition import PCA


In [12]:
mat_file = "dataset/PIE dataset/Pose05_64x64.mat"

data = sio.loadmat(mat_file)

data

{'__globals__': [],
 '__header__': 'MATLAB 5.0 MAT-file Platform: nt, Created on: Mon May 08 13:10:30 2017',
 '__version__': '1.0',
 'fea': array([[ 8,  9,  9, ..., 35, 36, 42],
        [ 6,  7,  7, ..., 38, 32, 31],
        [ 8,  8,  6, ..., 26, 25, 23],
        ..., 
        [11, 10,  8, ..., 36, 40, 42],
        [ 9, 11,  9, ..., 29, 31, 36],
        [12,  9,  9, ..., 29, 34, 38]], dtype=uint8),
 'gnd': array([[ 1],
        [ 1],
        [ 1],
        ..., 
        [68],
        [68],
        [68]], dtype=uint8),
 'isTest': array([[ 0.],
        [ 0.],
        [ 0.],
        ..., 
        [ 1.],
        [ 0.],
        [ 0.]])}

In [13]:
data['fea'].shape

(3332, 4096)

In [14]:
data['gnd'].shape

(3332, 1)

In [15]:
def consine_distance(feature1, feature2):
    '''
    @brief: compute consine distance between two vectors
    @param : feature1
    @param : feature2 
    '''
    num = np.dot(feature1,feature2.T)
    denom = np.linalg.norm(feature1) * np.linalg.norm(feature2)
    cos = num / denom #compute cos
    sim = 0.5 + 0.5 * float(cos)
    return sim

# ## test 
# a = np.array([1,1])
# b = np.array([-1,-1])
# print consine_distance(a,b)
        

In [19]:
def read_images ( path , sz = None ):
    c = 0
    X_train ,y_train = [] , []
    X_test ,y_test = [] , []
    for dirname , dirnames , filenames in os.walk ( path ):
        for subdirname in dirnames :
            subject_path = os.path.join ( dirname , subdirname )
            
            for filename in os.listdir ( subject_path ):
                if (filename != 'Thumbs.db'):
                    im = Image.open ( os.path.join ( subject_path , filename ))
                    im = im.convert ("L")
                    # resize to given size (if given )
                    if ( sz is not None ) :
                        im = im.resize (sz , Image.ANTIALIAS )
                        if (filename != '4.BMP' and filename != '8.BMP'):
                            X_train.append ( np.asarray (im , dtype = np.uint8 ) )
                            y_train.append (c)
                        else:
                            X_test.append ( np.asarray (im , dtype = np.uint8 ) )
                            y_test.append (c)
            c = c + 1
    return X_train , y_train, X_test, y_test

In [27]:
ROOT = "/Users/caishaofan/Desktop/IPython/HW_FaceVerification/dataset/ORL/ORL92112/bmp"
size = 23,28
X_train, y_train, X_test, y_test= read_images(ROOT, size)

In [28]:

X_train =  np.array(X_train)
y_train = np.array(y_train)
X_train_num = X_train.shape[0]

X_test = np.array(X_test)
y_test = np.array(y_test)
X_test_num = X_test.shape[0]

print X_train.shape
print X_test.shape
## reshape X_train X_test y_train y_test
X_train = X_train.reshape((X_train_num, -1))
X_test = X_test.reshape((X_test_num, -1))

print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(320, 28, 23)
(80, 28, 23)
(320, 644)
(80, 644)
(320,)
(80,)


In [41]:
pca = PCA(n_components = 50)

#transform the training data
X_train_reduced = pca.fit_transform(X_train)
#transform the test data
X_test_reduced = pca.transform(X_test)

print X_train_reduced.shape
print X_test_reduced.shape

(320, 50)
(80, 50)


In [42]:
print pca.explained_variance_ratio_

[ 0.19720308  0.14382384  0.07760078  0.06237401  0.05595871  0.03795961
  0.0280227   0.02611307  0.02166097  0.02014604  0.01609794  0.01565181
  0.01209159  0.01158073  0.01081075  0.00994615  0.00969912  0.00923012
  0.00825681  0.00755487  0.00717158  0.00703444  0.00644297  0.00587914
  0.00573102  0.00530072  0.00511894  0.00500027  0.00466684  0.00418095
  0.00408466  0.00401169  0.00398359  0.00374435  0.00369318  0.00357751
  0.00344304  0.00336379  0.00322935  0.00303899  0.00296013  0.0028838
  0.00281563  0.00269427  0.00258332  0.00241806  0.00239974  0.00227587
  0.0022049   0.00218611]


In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression()

#train
model.fit(X_train_reduced, y_train)
predictions = model.predict(X_test_reduced)

print classification_report(y_test, predictions)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00         2
          1       1.00      1.00      1.00         2
          2       1.00      1.00      1.00         2
          3       1.00      1.00      1.00         2
          4       1.00      1.00      1.00         2
          5       1.00      1.00      1.00         2
          6       1.00      1.00      1.00         2
          7       1.00      1.00      1.00         2
          8       1.00      1.00      1.00         2
          9       1.00      1.00      1.00         2
         10       1.00      1.00      1.00         2
         11       1.00      1.00      1.00         2
         12       1.00      1.00      1.00         2
         13       1.00      1.00      1.00         2
         14       1.00      1.00      1.00         2
         15       1.00      1.00      1.00         2
         16       1.00      1.00      1.00         2
         17       1.00      1.00      1.00   

In [151]:
print y_train

[ 0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  3
  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  6  6
  6  6  6  6  6  6  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  9  9  9
  9  9  9  9  9 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 12 12 12 12
 12 12 12 12 13 13 13 13 13 13 13 13 14 14 14 14 14 14 14 14 15 15 15 15 15
 15 15 15 16 16 16 16 16 16 16 16 17 17 17 17 17 17 17 17 18 18 18 18 18 18
 18 18 19 19 19 19 19 19 19 19 20 20 20 20 20 20 20 20 21 21 21 21 21 21 21
 21 22 22 22 22 22 22 22 22 23 23 23 23 23 23 23 23 24 24 24 24 24 24 24 24
 25 25 25 25 25 25 25 25 26 26 26 26 26 26 26 26 27 27 27 27 27 27 27 27 28
 28 28 28 28 28 28 28 29 29 29 29 29 29 29 29 30 30 30 30 30 30 30 30 31 31
 31 31 31 31 31 31 32 32 32 32 32 32 32 32 33 33 33 33 33 33 33 33 34 34 34
 34 34 34 34 34 35 35 35 35 35 35 35 35 36 36 36 36 36 36 36 36 37 37 37 37
 37 37 37 37 38 38 38 38 38 38 38 38 39 39 39 39 39 39 39 39]


In [44]:
for p,v in zip(y_test, predictions):
    print p, '\t', v, '\t', p == v

0 	0 	True
0 	0 	True
1 	1 	True
1 	1 	True
2 	2 	True
2 	2 	True
3 	3 	True
3 	3 	True
4 	4 	True
4 	4 	True
5 	5 	True
5 	5 	True
6 	6 	True
6 	6 	True
7 	7 	True
7 	7 	True
8 	8 	True
8 	8 	True
9 	9 	True
9 	9 	True
10 	10 	True
10 	10 	True
11 	11 	True
11 	11 	True
12 	12 	True
12 	12 	True
13 	13 	True
13 	13 	True
14 	14 	True
14 	14 	True
15 	15 	True
15 	15 	True
16 	16 	True
16 	16 	True
17 	17 	True
17 	17 	True
18 	18 	True
18 	18 	True
19 	19 	True
19 	19 	True
20 	20 	True
20 	20 	True
21 	21 	True
21 	21 	True
22 	22 	True
22 	22 	True
23 	23 	True
23 	23 	True
24 	24 	True
24 	24 	True
25 	25 	True
25 	25 	True
26 	26 	True
26 	26 	True
27 	27 	True
27 	27 	True
28 	28 	True
28 	28 	True
29 	29 	True
29 	29 	True
30 	30 	True
30 	30 	True
31 	31 	True
31 	31 	True
32 	32 	True
32 	32 	True
33 	33 	True
33 	33 	True
34 	34 	True
34 	34 	True
35 	35 	True
35 	35 	True
36 	36 	True
36 	36 	True
37 	37 	True
37 	37 	True
38 	38 	True
38 	38 	True
39 	39 	True
39 	39 	True


In [104]:
### PCA + SVM 
from sklearn.svm import LinearSVC
C_list = [0.000000115,0.00000015,0.0000001,0.00000001,0.000000089]
for c in C_list:
    model = LinearSVC(C= c, random_state= 42)

    model.fit(X_train_reduced, y_train)
    # model.decision_function_shape = "ovr"
    predictions = model.predict(X_test_reduced)
    print classification_report(y_test, predictions, digits = 4)

             precision    recall  f1-score   support

          0     1.0000    1.0000    1.0000         2
          1     1.0000    1.0000    1.0000         2
          2     1.0000    1.0000    1.0000         2
          3     1.0000    1.0000    1.0000         2
          4     1.0000    1.0000    1.0000         2
          5     1.0000    1.0000    1.0000         2
          6     1.0000    1.0000    1.0000         2
          7     1.0000    1.0000    1.0000         2
          8     1.0000    1.0000    1.0000         2
          9     1.0000    1.0000    1.0000         2
         10     1.0000    1.0000    1.0000         2
         11     1.0000    1.0000    1.0000         2
         12     1.0000    1.0000    1.0000         2
         13     1.0000    1.0000    1.0000         2
         14     1.0000    1.0000    1.0000         2
         15     1.0000    1.0000    1.0000         2
         16     1.0000    1.0000    1.0000         2
         17     1.0000    1.0000    1.0000   